## Підготовка даних

In [1]:
import pandas as pd

data = pd.read_csv('archive/duplicate_free_41K.csv')

In [31]:
from PIL import Image
import os
import numpy as np

image_folder = 'archive/img_41K/img_41K'
reference_image_path = os.path.join(image_folder, '0.jpg')

# Зчитуємо початкове зображення для отримання розміру
reference_image = Image.open(reference_image_path)
reference_size = reference_image.size

# Оголошуємо порожній масив images
images = []

for i in range(42021):
    filename = str(i) + '.jpg'
    file_path = os.path.join(image_folder, filename)
    if os.path.isfile(file_path):
        try:
            # Зчитуємо та перетворюємо зображення у розмір, що відповідає початковому зображенню
            image = Image.open(file_path).resize(reference_size)
            # Перетворюємо зображення в масив NumPy
            image_array = np.array(image)
            if len(image_array.shape) == 2:
                image_array = np.repeat(image_array[:, :, np.newaxis], 3, axis=2)
            # Додавання зображення до списку images
            images.append(image_array)
        except (IOError, OSError):
            print(f"Cannot process file: {file_path}")


In [ ]:
# Об'єднання масивів в один масив images за допомогою np.concatenate
images_array = np.array(images)

# Перевірка розміру масиву зображень
print(images_array.shape)

In [33]:
train_size = 29385
dev_size = 6297
test_size = 6297

train_x = images_array[:train_size+dev_size]
dev_x = images_array[train_size:train_size+dev_size]
test_x = images_array[train_size+dev_size:]

In [34]:
Y = data[data.columns[6:19]].values
train_y = Y[:train_size+dev_size]
dev_y = Y[train_size:train_size+dev_size]
test_y = Y[train_size+dev_size:]

## Створення моделі

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Визначення архітектури моделі
model = tf.keras.Sequential()

# Додавання згорткових шарів
model.add(layers.Conv2D(64, (3, 3), activation='softmax', input_shape=(386, 256, 3)))
model.add(layers.Conv2D(128, (3, 3), activation='sigmoid'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.AveragePooling2D(pool_size=(2, 2)))
# Зміна розмірності вхідних даних
model.add(layers.Flatten())

# Додавання повнозв'язаних шарів
model.add(layers.Dense(32, activation='sigmoid'))
model.add(layers.Dense(168, activation='tanh'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(13, activation='softmax'))

learning_rate = 0.00005
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

# Виведення структури моделі
model.summary()

## Тренування моделі

In [ ]:
for i in range(10):
    history = model.fit(train_x, train_y, epochs=1, batch_size=10)
    model.save('models/model' + str(i) + ':' + str(history.history['accuracy']) + '.h5')
    model_json = model.to_json()
    with open('models/model'  + str(i) + ':' + str(history.history['accuracy']) + '.json', 'w') as json_file:
        json_file.write(model_json)

In [ ]:
loss, accuracy = model.evaluate(test_x, test_y)

In [74]:
model.save('models/model1:[0.1673].h5')
model_json = model.to_json()
with open('models/model1:[0.1673].json', 'w') as json_file:
    json_file.write(model_json)